Different morphologies of the same neuron type might require different activation to reach comparable firing frequency. InputScaling simulates a network of a neuron type with different number of inputs, and different frequency to infer what level of input is needed.

Here we look at the input to ```neuron_types="fs"```.

During a simulation we can vary the input frequency, but not the number of inputs coupled to a neuron, so we use a set of ```num_replicas``` copies of each morphology, each receiving a different number of inputs from ```num_input_min``` to ```num_input_max``` of a given ```input_type``` (e.g. for striatum "cortical" or "thalamic"). 

A set of input frequencies are tested, specified by ```input_frequency_range```, and each of these frequencies is run for ```input_duration``` seconds. In this example we create a small example, but we recommend that use use a longer input duration (e.g. 10 seconds, or possibly more).

In [1]:
import os

bg_data_path = os.path.join("..","..","..", "BasalGangliaData", "data")
if os.path.isdir(bg_data_path):
    os.environ["SNUDDA_DATA"] = bg_data_path

from snudda.input.input_tuning import InputTuning
network_path = os.path.join("networks", "input_tuning")
input_tuning = InputTuning(network_path)

We create a network without any synapses and connect different number of external inputs to the neurons. Each neuron then receives a range of input frequencies. Here ```neurons_path``` points to the folder that contains all the different neuron types. Note that we use ```$DATA``` to refer to the ```snudda/data``` folder, but you can use an absolute path instead if you have your own neurons folder. If you use your own neurons you probably want to compile the neuron mechanisms from another folder when calling ```nrnivmodl``` below.

Also note that the folder names for the different neurons must be lowercase.

In [2]:
neurons_path = os.path.join("$DATA", "neurons", "striatum")
input_tuning.setup_network(neurons_path=neurons_path, 
                           num_replicas=5,
                           neuron_types="fs")
input_tuning.setup_input(input_type="cortical",  # eg. "cortical" or "thalamic"
                         num_input_min=100,
                         num_input_max=1000,
                         input_duration=2.0,
                         input_frequency_range=[1.0, 2.0, 3.0])

Skipping neuron type lts
Skipping neuron type dspn
Skipping neuron type ispn
Skipping neuron type chin
Found 5 neurons in /home/hjorth/HBP/BasalGangliaData/data/neurons/striatum/fs
Instantiates par_id = 0
morph_tag = DR-rat-Mar-13-08-1-536-R-cor-rep-res3-var0.swc
morph_tag = DR-rat-Mar-13-08-1-536-R-cor-rep-res3-var1.swc
morph_tag = DR-rat-Mar-13-08-1-536-R-cor-rep-res3-var2.swc
morph_tag = DR-rat-Mar-13-08-1-536-R-cor-rep-res3-var3.swc
morph_tag = DR-rat-Mar-13-08-1-536-R-cor-rep-res3-var4.swc
morph_tag = DR-rat-Mar-13-08-1-536-R-cor-rep-res3-var5.swc
morph_tag = DR-rat-Mar-13-08-1-536-R-cor-rep-res3-var6.swc
morph_tag = DR-rat-Mar-13-08-1-536-R-cor-rep-res3-var7.swc
morph_tag = DR-rat-Mar-13-08-1-536-R-cor-rep-res3-var8.swc
Instantiates par_id = 1
Instantiates par_id = 2
Instantiates par_id = 3
Instantiates par_id = 0
morph_tag = MTC251001A-IDB-cor-rep-res3-var0.swc
morph_tag = MTC251001A-IDB-cor-rep-res3-var1.swc
morph_tag = MTC251001A-IDB-cor-rep-res3-var2.swc
morph_tag = MTC251001

In [3]:
from snudda.input import SnuddaInput

si = SnuddaInput(input_config_file=input_tuning.input_config_file,
                 hdf5_network_file=input_tuning.network_file,
                 spike_data_filename=input_tuning.input_spikes_file,
                 time=input_tuning.max_time)
si.generate()

Writing spikes to networks/input_tuning/input.hdf5


## For large runs, to simulate in parallel use:

```mpiexec -n 4 python3 ../../snudda/input/input_tuning.py simulate networks/input_tuning/```

If you have more than four cores available on your machine, increase the number of workers accordingly.

## Below we just run it in serial in the notebook:

```mech_dir=None``` uses default mechanism directory in ```$SNUDDA_DATA/neurons/mechanisms```

In [4]:
input_tuning.simulate(mech_dir=None)

Running on master node:  nrnivmodl /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms
Could not find compiled mechanisms. Compile using 'nrnivmodl /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms' and retry simulation.


SystemExit: -1

/home/hjorth/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
input_tuning.analyse_results(show_plots=True)